# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql import Window
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import col
import pyspark.sql.functions as f

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [2]:
all_pages = user_log.select('page').dropDuplicates().collect()
unknown_user_pages = user_log.filter(user_log.userId == '').select('page').dropDuplicates().collect()
set(all_pages) - set(unknown_user_pages)

{Row(page='Downgrade'),
 Row(page='Error'),
 Row(page='Logout'),
 Row(page='NextSong'),
 Row(page='Save Settings'),
 Row(page='Settings'),
 Row(page='Submit Downgrade'),
 Row(page='Submit Upgrade'),
 Row(page='Upgrade')}

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


# Question 3

How many female users do we have in the data set?

In [3]:
user_log.groupBy('gender').agg(countDistinct("userId")).collect()

[Row(gender='F', count(DISTINCT userId)=462),
 Row(gender=None, count(DISTINCT userId)=1),
 Row(gender='M', count(DISTINCT userId)=501)]

In [4]:
user_log.filter(user_log.gender == 'F').groupBy('gender').agg(countDistinct("userId")).collect()

[Row(gender='F', count(DISTINCT userId)=462)]

# Question 4

How many songs were played from the most played artist?

In [5]:
user_log.filter(user_log.page == 'NextSong').groupBy('artist').count().orderBy(col('count').desc()).collect()

[Row(artist='Coldplay', count=83),
 Row(artist='Kings Of Leon', count=69),
 Row(artist='Florence + The Machine', count=52),
 Row(artist='BjÃ\x83Â¶rk', count=46),
 Row(artist='Dwight Yoakam', count=45),
 Row(artist='Justin Bieber', count=43),
 Row(artist='The Black Keys', count=40),
 Row(artist='OneRepublic', count=37),
 Row(artist='Jack Johnson', count=36),
 Row(artist='Muse', count=36),
 Row(artist='Radiohead', count=31),
 Row(artist='Taylor Swift', count=29),
 Row(artist='Lily Allen', count=28),
 Row(artist='Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', count=28),
 Row(artist='Train', count=28),
 Row(artist='Nickelback', count=27),
 Row(artist='Metallica', count=27),
 Row(artist='Daft Punk', count=27),
 Row(artist='Kanye West', count=26),
 Row(artist='John Mayer', count=24),
 Row(artist='Red Hot Chili Peppers', count=24),
 Row(artist='Alliance Ethnik', count=24),
 Row(artist='Linkin Park', count=23),
 Row(artist='Eminem', count=23),
 Row(artist="Guns N' Ros

In [6]:
user_log.filter(user_log.artist.isNotNull()).groupBy('artist').count().orderBy(col('count').desc()).collect()

[Row(artist='Coldplay', count=83),
 Row(artist='Kings Of Leon', count=69),
 Row(artist='Florence + The Machine', count=52),
 Row(artist='BjÃ\x83Â¶rk', count=46),
 Row(artist='Dwight Yoakam', count=45),
 Row(artist='Justin Bieber', count=43),
 Row(artist='The Black Keys', count=40),
 Row(artist='OneRepublic', count=37),
 Row(artist='Jack Johnson', count=36),
 Row(artist='Muse', count=36),
 Row(artist='Radiohead', count=31),
 Row(artist='Taylor Swift', count=29),
 Row(artist='Lily Allen', count=28),
 Row(artist='Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', count=28),
 Row(artist='Train', count=28),
 Row(artist='Nickelback', count=27),
 Row(artist='Metallica', count=27),
 Row(artist='Daft Punk', count=27),
 Row(artist='Kanye West', count=26),
 Row(artist='John Mayer', count=24),
 Row(artist='Red Hot Chili Peppers', count=24),
 Row(artist='Alliance Ethnik', count=24),
 Row(artist='Linkin Park', count=23),
 Row(artist='Eminem', count=23),
 Row(artist="Guns N' Ros

# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [7]:
function = udf(lambda user_home : int(user_home == 'Home'), IntegerType())

In [8]:
user_window = Window.partitionBy('userID').orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding, 0)

In [9]:
cusum = user_log.filter(((user_log.page == 'NextSong') | (user_log.page == 'Home')) & (user_log["userId"] != "") ) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

In [10]:
cusum.show()

+------+--------+-------------+---------+------+
|userID|    page|           ts|homevisit|period|
+------+--------+-------------+---------+------+
|  1436|NextSong|1513783259284|        0|     0|
|  1436|NextSong|1513782858284|        0|     0|
|  2088|    Home|1513805972284|        1|     1|
|  2088|NextSong|1513805859284|        0|     1|
|  2088|NextSong|1513805494284|        0|     1|
|  2088|NextSong|1513805065284|        0|     1|
|  2088|NextSong|1513804786284|        0|     1|
|  2088|NextSong|1513804555284|        0|     1|
|  2088|NextSong|1513804196284|        0|     1|
|  2088|NextSong|1513803967284|        0|     1|
|  2088|NextSong|1513803820284|        0|     1|
|  2088|NextSong|1513803651284|        0|     1|
|  2088|NextSong|1513803413284|        0|     1|
|  2088|NextSong|1513803254284|        0|     1|
|  2088|NextSong|1513803057284|        0|     1|
|  2088|NextSong|1513802824284|        0|     1|
|  2162|NextSong|1513781246284|        0|     0|
|  2162|NextSong|151

In [12]:
cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}).filter(col('count(period)') != 0).agg({'count(period)':'avg'}).collect()

[Row(avg(count(period))=6.898347107438017)]

In [15]:
cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}).agg({'count(period)':'max'}).collect()

[Row(max(count(period))=112)]